In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

pd.set_option("display.max_columns", None)

# User Data Frame

#### Goals

- Create function to read newest input survey and transform it to the same format as genres_recommender in order to append it and get the most similar streaming service.

Survey Link: https://forms.gle/Jb7gzK7VMbnvpRvP9

### 1. Import data

In [2]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"

survey = pd.read_csv(url)

We will remove the first 7 columns since they were used for testing the survey.

In [3]:
survey = survey.loc[7:].reset_index(drop=True)

survey

,Timestamp,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Quesiton 10,Question 11,Question 12,Question 13,Question 14,Question 15,Question 16,Question 17,Question 18,Quesiton 20,Question 19,Question 21,Question 22,Question 23,Question 24,Question 25,Question 26,Question 27,Question 28,Question 29,Question 30
0,5/15/2020 11:50:06,Revisions (2019) - When Shibuya time-warps to ...,Big Mouth (2017) - Teenage friends find their ...,American Horror Story (2011) - An anthology se...,Chilling Adventures of Sabrina (2018) - As her...,Hart of Dixie (2011) - New Yorker and new doct...,Black Mirror (2011) - An anthology series expl...,Stranger Things (2016) - When a young boy disa...,Crazy Ex-Girlfriend (2015) - A young woman aba...,Supernatural (2005) - Two brothers follow thei...,Fauda (2015) - The human stories on both sides...,"The Witcher (2019) - Geralt of Rivia, a solita...",Cheer (2020) - Follows the cheerleaders of Nav...,Ozark (2017) - A financial advisor drags his f...,"Tiger King: Murder, Mayhem and Madness (2020) ...",Yeh Meri Family (2018) - Set in the summer of ...,Five Came Back (2017) - The wartime contributi...,The Circle (2020) - Players start off isolated...,Rythm + Flow (2019) - Global superstars Cardi ...,The Motorbike Show (2011) - It's a show which ...,The 100 (2014) - Set ninety-seven years after ...,"Don't Watch This (2018) - Ghoulish creatures, ...",Blue Mountain State (2010) - Three incoming fr...,The Curious Creations of Christine McConnell (...,The Vietnam War (2017) - A comprehensive histo...,The Ranch (2016) - The son of a Colorado ranch...,The End of the F***ing World (2017) - James is...,Once Upon a Time (2011) - A young woman with a...,The Flash (2014) - After being struck by light...,BoJack Horseman (2014) - BoJack Horseman was t...,"Love, Death & Robots (2019) - A collection of ..."
1,5/15/2020 11:55:34,Survivor (2000) - A reality show where a group...,The Amazing Race (2001) - Multiple teams race ...,Klondike (2014) - The lives of two childhood b...,The Planets (2019) - Welcome to the greatest s...,The Joy of Painting (1983) - In this half-hour...,Eat. Race. Win (2018) - EAT. RACE. WIN. is a b...,Generation Kill (2008) - A Rolling Stone repor...,Bonanza (1959) - The Wild West adventures of B...,How The Universe Works (2010) - A users guide ...,Strictly Ballet (2014) - Join us as we meet Ja...,Baseball (1994) - A documentary on the history...,The Boys (2019) - A group of vigilantes set ou...,Teen Wolf (2011) - An average high school stud...,Chuck (2007) - When a twenty-something compute...,Psycho-Pass (2012) - Believing in humanity and...,The Durrells (2016) - Based on the real-life ...,Fortitude (2015) - Perched on the edge of the ...,"Mozart in the Jungle (2014) - Love, money, amb...",The Man in the High Castle (2015) - In a dysto...,"The Young Turks (2005) - A daring, funny, and ...",Modern Love (2019) - TV series based on the Ne...,The Rifleman (1958) - The adventures of a Wild...,Just Add Magic (2015) - Three friends cook up ...,The Hitchhiker's Guide to the Galaxy (1981) - ...,Hell's Kitchen (2005) - World renowned chef Go...,Horrible Histories (2009) - A group of British...,Poldark (2015) - Ross Poldark returns home aft...,Good Omens (2019) - A tale of the bungling of ...,Undone (2019) - A woman discovers she has a ne...,The Snowy Day (2016) - Based on the award-winn...
2,5/15/2020 11:56:51,John Adams (2008) -The life of one of the USA'...,Sharp Objects (2018) - A reporter confronts th...,The Sopranos (1999) - New Jersey mob boss Tony...,Vinyl (2016) - A New York music executive in t...,Ballers (2015) - A series centered around a gr...,Any Given Wednesday with Bill Simmons (2016) -...,Silicon Valley (2014) - Follows the struggle o...,The Night Of (2016) - After a night of partyin...,Last Week Tonight with John Oliver (2014) - Fo...,VICE News Tonight (2016) - A nightly news show...,Westworld (2016) - Set at the intersection of ...,The Leftovers 

Rows 1-3 were used to test picking only Netflix shows (first row), then Amazon shows (second row) and finally HBO shows (third row). We will start working with the first row, in order to test Netflix and later on we will create a function for Amazon and HBO using the same process.

We will also need to join all streaming services genres data frames in order to obtain the genres data frame for our survey users.

In [4]:
netflix = pd.read_pickle("../Data/netflix_genres.pkl")
amazon = pd.read_pickle("../Data/amazon_genres.pkl")
hbo = pd.read_pickle("../Data/hbo_genres.pkl")

In [5]:
all_shows = netflix.append(amazon).append(hbo).reset_index(drop=True)

In [6]:
all_shows.to_pickle("../Data/all_shows_genres.pkl")

### 2. Remove Timestamp column

We will remove the Timestamp column since it would not be useful for our analysis.

In [7]:
survey = survey.drop(columns="Timestamp")

### 3. Get list of TV shows for survey user

In [8]:
all_shows_lst = [show for show in all_shows["show"]]

In [9]:
# using regex, we will get the title of TV shows selected

titles_netflix = []

for show in survey.loc[0].values:
    # remove the year and description
    title_pattern = r"(.*)\s\(\d{4}\).*$"
    title = re.findall(title_pattern, show)
    titles_netflix.append(title[0])


matches_netflix = []

for title in titles_netflix:
    for show in all_shows_lst:
        
        # compute ratio and remove case-sensitivity
        ratio = fuzz.ratio(title.lower(), show.lower())

        # add titles to list with ratio = 100
        if ratio == 100:
            matches_netflix.append(show)
            
            # we will remove duplicates since there might be shared TV shows between streaming services
            matches_netflix = list(set(matches_netflix))

matches_netflix

['The Witcher',
 'Crazy Ex-Girlfriend',
 'Supernatural',
 'Once Upon a Time',
 'Blue Mountain State',
 'Black Mirror',
 'Big Mouth',
 'Fauda',
 'Tiger King: Murder, Mayhem and Madness',
 'Cheer',
 'Chilling Adventures of Sabrina',
 "Don't Watch This",
 'The Curious Creations of Christine McConnell',
 'The Circle',
 'The End of the F***ing World',
 'The 100',
 'Hart of Dixie',
 'The Vietnam War',
 'BoJack Horseman',
 'Five Came Back',
 'Yeh Meri Family',
 'Love, Death & Robots',
 'Ozark',
 'Stranger Things',
 'revisions',
 'American Horror Story',
 'The Flash',
 'The Motorbike Show',
 'The Ranch']

It works, now we will get the data frame for the shows listed on matches_netflix.

### 4. Create data frame to use for recommender model

In [10]:
user_netflix = all_shows[all_shows["show"].isin(matches_netflix)].reset_index(drop=True)

user_netflix

,show,genres,Crime,Drama,Thriller,Fantasy,Horror,Mystery,Comedy,Sci-Fi,Biography,Action,Adventure,Romance,History,Documentary,Animation,War,Sport,Family,Western,Short,Reality-TV,Musical,Music,Game-Show,Talk-Show,News
0,Stranger Things,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
1,Black Mirror,"Drama,Sci-Fi,Thriller",0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
2,Ozark,"Crime,Drama,Thriller",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
3,Supernatural,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
4,The Witcher,"Action,Adventure,Drama",0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
5,The Flash,"Action,Adventure,Drama",0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
6,The End of the F***ing World,"Adventure,Comedy,Crime",1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
7,The 100,"Drama,Mystery,Sci-Fi",0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
8,"Tiger King: Murder, Mayhem and Madness","Crime,Documentary",1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0,0,0,0,0,0
9,BoJack Horseman,"Animation,Comedy,Drama",0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0,0,0,0,0,0


We will follow the same process as done on 05_genres and 06.1_streaming_service_recommender_df

In [11]:
genre_columns = [col for col in user_netflix.columns if col not in ["show", "genres"]]

In [12]:
user_netflix_ratio = (pd.DataFrame(user_netflix[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

user_netflix_ratio["ratio"] = round((user_netflix_ratio["frequency"] / 30), 4)

user_netflix_ratio["user"] = "User"

user_netflix_recommender = user_netflix_ratio.pivot_table(index="user",
                                      columns="genre",
                                      values="ratio")

In [13]:
user_netflix_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User,0.1333,0.1333,0.1333,0.0333,0.3333,0.1,0.1333,0.5,0.0333,0.1667,0.0333,0.0667,0.2333,0.0333,0.0333,0.0333,0.0333,0.0667,0.1,0.1,0.0333,0.0333,0.0333,0.1667,0.0333,0.0333


Now we have our final dataframe, we will now create a function that sums up all of this steps.

### 5. Create function

In [14]:
# the input of this function will be the df survey, all_shows df, genre_columns and row location

def get_recommender_df(survey_df, df_all_shows, genre_columns, row):
    
    survey_df = survey_df.drop(columns="Timestamp")
    
    all_shows_lst = [show for show in df_all_shows["show"]]
    
    titles = []

    for show in survey_df.loc[row].values:
        # remove the year and description
        title_pattern = r"(.*)\s\(\d{4}\).*$"
        title = re.findall(title_pattern, show)
        titles.append(title[0])

    matches = []

    for title in titles:
        for show in all_shows_lst:
            ratio = fuzz.ratio(title.lower(), show.lower())
            if ratio == 100:
                matches.append(show)
                matches = list(set(matches))
                              
    user_df = df_all_shows[df_all_shows["show"].isin(matches)].reset_index(drop=True)
    
    user_genre_ratio = (pd.DataFrame(user_df[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

    user_genre_ratio["ratio"] = round((user_genre_ratio["frequency"] / 30), 4)

    user_genre_ratio["user"] = "User"

    user_recommender = user_genre_ratio.pivot_table(index="user",
                                          columns="genre",
                                          values="ratio")
    
    return user_recommender

In order to test the function, we will import everything needed again.

In [15]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"

survey_test = pd.read_csv(url)
survey_test = survey_test.loc[7:].reset_index(drop=True)

all_shows_test = pd.read_pickle("../Data/all_shows_genres.pkl")

genre_columns = [col for col in all_shows_test.columns if col not in ["show", "genres"]]

In [16]:
user_amazon_recommender = get_recommender_df(survey_test, all_shows_test, genre_columns, 1)

In [17]:
user_amazon_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User,0.1333,0.1333,0.1,0.0333,0.3333,0.0667,0.1667,0.3667,0.2,0.1333,0.1,0.1,0.0333,0.0333,0.0,0.0333,0.0333,0.1,0.0667,0.0667,0.0667,0.0333,0.0333,0.0333,0.0667,0.1


In [18]:
user_hbo_recommender = get_recommender_df(survey_test, all_shows_test, genre_columns, 2)

In [19]:
user_hbo_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,,,,,,,,
User,0.0667,0.0333,0.0333,0.1,0.5,0.3667,0.1333,0.7333,0.0333,0.1,0.0,0.1333,0.0333,0.1,0.0667,0.2,0.0667,0.0333,0.1333,0.0333,0.0,0.1333,0.0667,0.0333,0.0,0.0


It works, now we will export our data to test our model.

### 6. Export data

In [20]:
# user_netflix_recommender.to_pickle("../Data/netflix_test.pkl")

# user_amazon_recommender.to_pickle("../Data/amazon_test.pkl")

# user_hbo_recommender.to_pickle("../Data/hbo_test.pkl")

### 7. Update function

For our recommender, we will need always the last row from our survey data frame, so we will update the function in order to get just the last column. We will also have the genre_columns inside the function.

In [21]:
def get_recommender_df(survey_df, df_all_shows):
    
    genre_columns = ['Crime', 'Drama', 'Thriller', 'Fantasy', 'Horror', 'Mystery', 'Comedy', 'Sci-Fi', 'Biography',
                 'Action', 'Adventure', 'Romance', 'History', 'Documentary', 'Animation', 'War', 'Sport',
                 'Family', 'Western', 'Short', 'Reality-TV', 'Musical', 'Music', 'Game-Show', 'Talk-Show', 'News']
    
    survey_df = survey_df.drop(columns="Timestamp")
    
    all_shows_lst = [show for show in df_all_shows["show"]]
    
    titles = []
        
    # we will select just the last element using the tail pandas method
    for show in survey_df.tail(1).values[0]:
        # remove the year and description
        title_pattern = r"(.*)\s\(\d{4}\).*$"
        title = re.findall(title_pattern, show)
        titles.append(title[0])

    matches = []

    for title in titles:
        for show in all_shows_lst:
            ratio = fuzz.ratio(title.lower(), show.lower())
            if ratio == 100:
                matches.append(show)
                matches = list(set(matches))
                              
    user_df = df_all_shows[df_all_shows["show"].isin(matches)].reset_index(drop=True)
    
    user_genre_ratio = (pd.DataFrame(user_df[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

    user_genre_ratio["ratio"] = round((user_genre_ratio["frequency"] / 30), 4)

    user_genre_ratio["user"] = "User"

    user_recommender = user_genre_ratio.pivot_table(index="user",
                                          columns="genre",
                                          values="ratio")
    
    return user_recommender